In [37]:
from pymongo import MongoClient
import pandas as pd
import os
from dotenv import load_dotenv


In [38]:
# Load environment variables from .env file
load_dotenv()

# Get the DATABASE_URI, DATABASE_NAME, and COLLECTION_NAME from environment variables
database_uri = os.getenv('DATABASE_URI')
db_name = os.getenv('DB_NAME')
humans = os.getenv('DB_COLLECTION_1')
mod_bot = os.getenv('DB_COLLECTION_2')
adv_bot = os.getenv('DB_COLLECTION_3')

# Connect to MongoDB using the URI from environment variable
client = MongoClient(database_uri)

# Create or switch to your desired database
db = client[db_name]


In [39]:
# Fetch data from the collection
humans_mouse_movements = db[humans]
mod_bot_mouse_movements = db[mod_bot]
adv_bot_mouse_movements = db[adv_bot]

In [47]:
humans_cursor = humans_mouse_movements.find()
mod_bot_cursor = mod_bot_mouse_movements.find()
adv_bot_cursor = adv_bot_mouse_movements.find()

In [ ]:
humans_df = pd.DataFrame(list(humans_cursor))
mod_bot_df = pd.DataFrame(list(mod_bot_cursor))
adv_bot_df = pd.DataFrame(list(adv_bot_cursor))

In [48]:
humans_df

,_id,session_id,mousemove_client_height_width,mousemove_times,mousemove_total_behaviour,mousemove_visited_urls,unique_id
0,5f0221209dbc2279e9e1db9d,koftcfkosigp4f2ibgfi0dm0co,"[(938,1920)][(938,1920)][(938,1920)][(938,1920...","1593975071137,1593975071228,1593975071247,1593...","[m(456,701)][m(451,700)][m(436,707)][m(414,718...",[http://160.40.52.164/][http://160.40.52.164/]...,694842305985
1,5f0381d39dbc2279e9e23c10,89vshcu5jgp79ebb68f7jjkqc5,"[(938,1920)][(938,1920)][(938,1920)][(938,1920...","1594065360371,1594065360387,1594065360404,1594...","[m(665,326)][m(669,339)][m(678,370)][m(694,406...",[http://160.40.52.164/][http://160.40.52.164/]...,694842305985
2,5f12cf7cb8f8f8d39f6b9a9b,cmtn6a23es1vk1v038j2pd0cqa,"[(966,1920)][(966,1920)][(966,1920)][(966,1920...","1595068283506,1595068283521,1595068283538,1595...","[m(461,1)][m(463,3)][m(463,4)][m(462,4)][m(461...",[http://160.40.52.164/][http://160.40.52.164/]...,905889798372
3,5f1acb48af162dbbf9dbee34,8is1s1cv7tu0c13vcqka04ck8m,"[(966,1920)][(966,1920)][(966,1920)][(966,1920...","1595149169521,1595149169537,1595149169553,1595...","[m(1107,43)][m(1106,43)][m(1105,43)][m(1086,43...",[http://160.40.52.164/][http://160.40.52.164/]...,906446501406
4,5f1acb6daf162dbbf9dbee35,ad9scp2lb57iibuqakamfu5hjg,"[(601,1349)][(601,1349)][(601,1349)][(601,1349...","1595231165305,1595231165315,1595231165320,1595...","[m(322,4)][m(329,14)][m(334,21)][m(341,30)][m(...",[http://160.40.52.164/][http://160.40.52.164/]...,150084307074
5,5f1acbcaaf162dbbf9dbee37,7a9emfkn0ri8ntg4k0uotsi07v,"[(978,1858)][(978,1858)][(978,1858)][(978,1858...","1595259367864,1595259367896,1595259367928,1595...","[m(914,7)][m(885,66)][m(870,98)][m(854,127)][m...",[http://160.40.52.164/][http://160.40.52.164/]...,164032122012
6,5f1acc05af162dbbf9dbee38,oq11t11ns4t2gj9cdnl10da80s,"[(722,1536)][(722,1536)][(722,1536)][(722,1536...","1595312539852,1595312539867,1595312539884,1595...","[m(278,1)][m(278,10)][m(278,19)][m(278,26)][m(...",[http://160.40.52.164/][http://160.40.52.164/]...,650764971435
7,5f1acc43af162dbbf9dbee3a,gt6ccjle54o9oqntbbt3vci5pa,"[(969,1920)][(969,1920)][(969,1920)][(969,1920...","1595421152833,1595421152836,1595421152843,1595...","[m(434,968)][m(446,962)][m(469,947)][m(487,934...",[http://160.40.52.164/][http://160.40.52.164/]...,627535178922
8,5f1acc5caf162dbbf9dbee3b,0uhr32hgcllradm0p6b15371gi,"[(639,1354)][(639,1354)][(639,1354)][(639,1354...","1595499336625,1595499336642,1595499336657,1595...","[m(451,58)][m(453,59)][m(454,60)][m(454,61)][m...",[http://160.40.52.164/][http://160.40.52.164/]...,309927267811
9,5f29b7512265de3c42722733,p7g6ccuqqfum0pbfk5q82nhnam,"[(689,1263)][(689,1263)][(689,1263)][(689,1263...","1596534026465,1596534026481,1596534026498,1596...","[m(283,149)][m(286,149)][m(294,153)][m(316,169...",[http://160.40.52.164/][http://160.40.52.164/]...,18862862784
